In [5]:
import tempfile
import shutil
from pathlib import Path

In [10]:
mkdir test_dir

In [33]:
%%writefile test_dir/i_will_break.py
f = open('test_dir/poop', 'w')
f.write('sfaregwrhwrhh')
1 / 0 

Overwriting test_dir/i_will_break.py


In [37]:
code = (target_dir / 'i_will_break.py').read_text()
exec(code)

ZeroDivisionError: division by zero

In [39]:
backup_dir = Path(tempfile.mkdtemp()) / 'backup'
target_dir = Path('test_dir/')

In [40]:
shutil.copytree(target_dir, backup_dir)

PosixPath('/var/folders/vr/c6gyty1d7rb_swzt_j31r4rdg1hszv/T/tmpq_slj50_/backup')

In [41]:
ls /var/folders/vr/c6gyty1d7rb_swzt_j31r4rdg1hszv/T/tmpq_slj50_/backup

i_will_break.py


In [38]:
try:
    code = (target_dir / 'i_will_break.py').read_text()
    exec(code)
except Exception as e:
    print("The following error occurs during templating, Rolling back to last stable state.")
    print(e)
    shutil.rmtree(target_dir)
    shutil.copytree(backup_dir, target_dir)
    print(f'Copied {backup_dir} to {target_dir}')

The following error occurs during templating, Rolling back to last stable state.
division by zero
Copied /var/folders/vr/c6gyty1d7rb_swzt_j31r4rdg1hszv/T/tmp4fqvwzfd/backup to test_dir


In [ ]:
def safe_render(old, new, context):
    """Safely Render a new template by first making a backup to roll back to if needed."""

    # Copy current directory to temp backup location
    backup_dir = Path(tempfile.mkdtemp()) / 'backup'
    shutil.copytree(target_dir, backup_dir)

    try:
        output_dir, content = render_cookiecutter(
            template, no_input=True,
            extra_context=context,
            output_dir=target_dir,
            overwrite_if_exists=True,
        )
    except Exception as e:
        # Rollback
        print("The following error occurs during templating, Rolling back to last stable state.")
        print(e)
        shutil.copytree(backup_dir, target_dir)
        shutil.rmtree(target_dir)
        print('Copied {backup_dir} to {target_dir.parent}')
        sys.exit(1)

    shutil.rmtree(backup_dir.as_posix())

    return output_dir, content